In [1]:
# Instala y actualiza silenciosamente langchain, langchain-community y langchain-chroma
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [14]:
# LangSmith : Configura variables de entorno para rastreo y autenticación
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

In [15]:
# Configura la clave de API de OpenAI para acceder a sus modelos
%pip install -qU langchain-openai
# Instala BeautifulSoup para analizar HTML
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hannj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hannj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [51]:
# Instancia del modelo ChatOpenAI GPT-4 mini
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [52]:
# Carga y divide el contenido de un blog en fragmentos para indexación
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Carga contenido web y extrae solo elementos relevantes
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Divide el documento en fragmentos de 1000 caracteres con superposición de 200
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Crea un vector store e indexa los fragmentos
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Configura la recuperación y generación usando los fragmentos indexados
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


# Formatea los fragmentos recuperados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Configura la cadena de recuperación y generación (RAG) para responder preguntas
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# Ejecuta una pregunta de ejemplo usando la cadena RAG
rag_chain.invoke("What is Task Decomposition?")

c:\Users\hannj\AppData\Local\Programs\Python\Python39\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Task Decomposition is a process where complex tasks are broken down into smaller, more manageable steps to enhance performance and understanding. Techniques like Chain of Thought (CoT) guide models to think step by step, while methods like Tree of Thoughts explore multiple reasoning possibilities at each step. This can be achieved through various prompting techniques, task-specific instructions, or human inputs.'

In [53]:
# en la base de datos de vectores limpia y elimina la colección
vectorstore.delete_collection()

In [54]:
# Extrae datos específicos de la página web
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Conserve únicamente el título de la publicación, los encabezados y el contenido del HTML completo.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

# Muestra la longitud del contenido
len(docs[0].page_content)

43131

In [55]:
# Imprime los primeros 500 caracteres del contenido cargado para verificar
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [56]:
# Configura un divisor de texto que incluye índices de inicio
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# Muestra la cantidad de fragmentos
len(all_splits)

66

In [57]:
# Muestra la longitud del contenido del primer fragmento
len(all_splits[0].page_content)

969

In [58]:
# Muestra los metadatos del décimo fragmento
all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 7056}

In [59]:
# Crea y almacena los fragmentos en una base de datos de vectores
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [60]:
# Configura el recuperador y realiza una consulta de ejemplo
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

# Muestra la cantidad de documentos recuperados
len(retrieved_docs)

6

In [61]:
# Imprime el contenido del primer documento recuperado
print(retrieved_docs[0].page_content)

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


In [62]:
# Actualiza nuevamente la biblioteca langchain-openai
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\hannj\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [63]:
# Configura OpenAI nuevamente y crea el modelo GPT-4 mini
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [64]:
# Configura y ejecuta un prompt de ejemplo usando el prompt hub
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

c:\Users\hannj\AppData\Local\Programs\Python\Python39\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [65]:
# Imprime el contenido del mensaje de ejemplo
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [66]:
# Configura una cadena RAG con plantillas y respuesta a preguntas
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. This approach helps in planning and executing complex tasks efficiently, allowing for better organization and clarity in the problem-solving process. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by enabling structured reasoning and exploration of multiple possibilities at each step.


In [67]:
# Imprime los documentos fuente en respuesta
for document in response["context"]:
    print(document)
    print()

page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1585}

page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS 

In [68]:
# Configura y ejecuta un prompt personalizado con mensaje de cortesía
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is the process of breaking down a complex task into smaller, manageable steps to facilitate easier execution and understanding. Techniques like Chain of Thought (CoT) and Tree of Thoughts extend this concept by structuring the reasoning process and exploring multiple possibilities at each step. This approach enhances model performance and allows for clearer insight into the model's thinking. Thanks for asking!"